# Day 1 - Setting up your system

## Activity 0: Installing Ubuntu (15.10, 64-bit)

Installation of Ubuntu is sufficiently complicated to not be covered here. We'll proceed with the assumption that you already have a machine with pre-installed Ubuntu at your disposal, or you know someone who can help you do it. This is one of the few major assumptions we'll make. [Here is the official documentation](https://help.ubuntu.com/community/Installation).

The 64-bit 15.10 edition of Ubuntu is the recommended one. Almost all the code that follow will work with almost no tweakings for other versions of Ubuntu as well. Particularly, wherever you see **wily** in bash commands, you should replace it with the appropriate name for the version of Ubuntu you are going to use (for example, *trusty* for 14.04 or *vivd* for 15.04. The full list can be found [here](https://en.wikipedia.org/wiki/List_of_Ubuntu_releases).).

You also need to be comfortable using Ubuntu as your operating system. In following sections, we cover a list of ideas and tasks that you should have an understanding of in order to be able to use Ubuntu:

### Mastering Ubuntu

* You should understand the basics of terminal-based manipulation of the operating environment:
  - Is, rm, mv, mkdir, cp
  - top
  - sudo
  - Hidden files
  - An understanding of the file structure (including their home directory)
* You should be able to understand the login process and the way environmental variables are set:
  - .profile
  - .bashrc
  - .bash_history
* You should understand how package management works via apt-get (CLI) and synaptic (GUI):
  - Updating package list (apt-get update)
  - Upgrading to new packages (apt-get upgrade)
  - Adding new repositories to apt (and adding secure keys using apt-key)
  - Adding PPAs
  - Finding and installing new softwares
  - Installing software from .deb files (dpkg -i filename.deb) - walk through the process of installing google chrome (not chromium)
* Introduction to basic tools
  - gedit
  - LibreOffice Calc
  - Changing System Settings
  - Change desktop behaviour
  - Create and set up new users
  - System Monitor (and what do the various components mean)

All most all the topics mentioned above (and a little more) can be found in the following links (strongly suggested that you read these end-to-end if the material is new for you):

* Using the terminal - https://help.ubuntu.com/community/UsingTheTerminal
* Commands on the terminal - https://help.ubuntu.com/community/CommandlineHowto
* Apt-Get Howto - https://help.ubuntu.com/community/AptGet/Howto
* (Optional on Day 1) Deeper into Apt-Get - https://help.ubuntu.com/community/Repositories/CommandLine

## Activity 1: Setting up Ubuntu

The software in the system is likely to have become update. Let's update the software in our system (note: all the bash commands are meant to be run on the terminal, but you already know that by now).

```bash
sudo apt-get update
sudo apt-get -y upgrade
```

Now we'll go ahead and install some packages that we'll need from time to time (more so if we are going to be using Ubuntu as our primary operating system). Please check the licensing laws in your country for the softwares included in the *restricted* packages below before instaling.

```bash
sudo apt-get install -y software-properties-common curl
sudo apt-get install default-jre default-jdk
sudo apt-get install ubuntu-restricted-extras ubuntu-restricted-addons
```

Set Timezone

```bash
sudo ln -sf /usr/share/zoneinfo/UTC /etc/localtime
```

**Optional on Day 1**

More advanced/adventurous students who wish to install the jdk/jre from Oracle (if you don't know what I am talking about, don't venture any further) can execute the following:

```bash
sudo apt-get install python-software-properties
sudo add-apt-repository ppa:webupd8team/java
sudo apt-get update

sudo apt-get install oracle-java8-installer
```

If the Oracle version is the only one you have installed, then you have only one Java framework installed i your system. Otherwise, if you have multiple Java frameworks installed in your system, you can choose one of them with the following:

```bash
sudo update-alternatives --config java
sudo update-alternatives --config javac
```

And here is how you can set up your JAVA_HOME

* Note in the install path from (the command above)
        sudo update-alternatives --config java
* Add JAVA_HOME="YOUR_PATH" to /etc/environment
        source /etc/environment
        echo $JAVA_HOME

## Activity 2: Installing SQL Server

### Setting up PostgreSQL

```bash
sudo apt-get install -y postgresql-9.4 postgresql-contrib-9.4 postgresql-server-dev-all

# Configure Postgres Remote Access
sudo sed -i "s/#listen_addresses = 'localhost'/listen_addresses = '*'/g" /etc/postgresql/9.4/main/postgresql.conf
sudo echo "host    all             all             10.0.2.2/32               md5" | tee -a /etc/postgresql/9.4/main/pg_hba.conf
sudo -u postgres psql -c "CREATE ROLE iamsrk LOGIN UNENCRYPTED PASSWORD 'iamsrk123' SUPERUSER INHERIT NOCREATEDB NOCREATEROLE NOREPLICATION;"
sudo -u postgres /usr/bin/createdb --echo --owner=iamsrk testdb
sudo service postgresql restart
```

Now you have

* A working PostgreSQL server
* A user called *iamsrk* with password *iamsrk123*
* A database owned by *iamsrk* called *testdb*

### Setting up MySQL

**This section needs a little more work. Should be skipped for the time being**

```bash
sudo apt-key adv --keyserver ha.pool.sks-keyservers.net --recv-keys 5072E1F5
sudo sh -c 'echo "deb http://repo.mysql.com/apt/ubuntu/ trusty mysql-5.6" >> /etc/apt/sources.list.d/mysql.list'

sudo debconf-set-selections <<< "mysql-community-server mysql-community-server/data-dir select ''"
sudo debconf-set-selections <<< "mysql-community-server mysql-community-server/root-pass password secret"
sudo debconf-set-selections <<< "mysql-community-server mysql-community-server/re-root-pass password secret"
sudo apt-get install -y mysql-server

# Configure MySQL Password Lifetime
sudo echo "default_password_lifetime = 0" >> /etc/mysql/my.cnf

# Configure MySQL Remote Access
sudo sed -i '/^bind-address/s/bind-address.*=.*/bind-address = 0.0.0.0/' /etc/mysql/my.cnf

mysql --user="root" --password="secret" -e "GRANT ALL ON *.* TO root@'0.0.0.0' IDENTIFIED BY 'secret' WITH GRANT OPTION;"
sudo service mysql restart

mysql --user="root" --password="secret" -e "CREATE USER 'iamsrk'@'0.0.0.0' IDENTIFIED BY 'iamsrk123';"
mysql --user="root" --password="secret" -e "GRANT ALL ON *.* TO 'iamsrk'@'0.0.0.0' IDENTIFIED BY 'iamsrk123' WITH GRANT OPTION;"
mysql --user="root" --password="secret" -e "GRANT ALL ON *.* TO 'iamsrk'@'%' IDENTIFIED BY 'iamsrk123' WITH GRANT OPTION;"
mysql --user="root" --password="secret" -e "FLUSH PRIVILEGES;"
mysql --user="root" --password="secret" -e "CREATE DATABASE testdb;"
sudo service mysql restart

# Add Timezone Support To MySQL
mysql_tzinfo_to_sql /usr/share/zoneinfo | mysql --user=root --password=secret mysql
```

Now you have

* A working MySQL server
* A user called *iamsrk* with password *iamsrk123*
* *root* password is now *secret*
* There is a database called *testdb* that *iamsrk* has access to

## Activity 3: Installing NoSQL Server

**This section needs a little more work. Should be skipped for the time being**

### Setting up MongoDB

## Activity 4: Installing R

```bash
echo R_LIBS_USER="~/bin/R/library" >> ~/.Renviron
# Add default repo here ---
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys E084DAB9
sudo sh -c 'echo "deb https://cran.rstudio.com/bin/linux/ubuntu wily/" >> /etc/apt/sources.list.d/cranR.list'

sudo apt-get update
sudo apt-get install -y r-base r-base-dev r-recommended

# In R
sudo apt-get install libcurl4-openssl-dev #for Rcurl
Rscript -e 'install.packages(c("ggplot2", "data.table", "RPostgreSQL", "dplyr"), repos="http://cran.rstudio.com/")'
```

## Activity 5: Installing Python

* [What is Anaconda?](https://en.wikipedia.org/wiki/Anaconda_%28Python_distribution%29)
* [Here](http://docs.python-guide.org/en/latest/scenarios/scientific/) is an introduction to the scientific computing stack in Python. You may wish to [navigate](http://docs.python-guide.org/en/latest/scenarios/scientific/#anaconda) to the section on Anaconda.

### Installing Anaconda

* Download: https://www.continuum.io/downloads
    
Distribution | Link
------------ | -----
Linux 64-bit: | https://3230d63b5fc54e62148e-c95ac804525aac4b6dba79b00b39d1d3.ssl.cf1.rackcdn.com/Anaconda-2.3.0-Linux-x86_64.sh
Linux 32-bit: | https://3230d63b5fc54e62148e-c95ac804525aac4b6dba79b00b39d1d3.ssl.cf1.rackcdn.com/Anaconda-2.3.0-Linux-x86.sh

* Install
```bash
bash Anaconda-2.3.0-Linux-x86_64.sh
```

### Testing Anaconda Install

* Verifying Anaconda install
```bash
conda --version
```
* Updating Anaconda to current version
```bash
conda update conda
```
* List installed packages
```bash
conda list
```
* Search for a package
```bash
conda search beautifulsoup4
```
* Install packages
```bash
conda install beautifulsoup4
pip install beautifulsoup4
pip install ipython
```
* Remove packages
```bash
conda remove beautifulsoup4
```

### IPython Basics

* [Get started with IPython](https://www.youtube.com/watch?v=j08Ry-8MOxM) (ignore the sections on installation)
* You should understand the following concepts:
  - Managing Cells
     * Cell Types
     * Moving Cells around
     * Executing and Creating New Cells
  - Handling Notebooks
     * Creating new notebooks (with different kernels)
     * Exporting notebooks to various formats
  - IPython Kernels
  - Keyboard Shortcuts

IPython Homepage: https://ipython.org/

### Installing R Kernel (IRkernel)

```bash
sudo apt-get install libzmq3-dev
```
(note: if you are using an older version of Ubuntu, you may want to install *libzmq-dev* instead if you are getting errors below.)

The following code should be executed inside an R session (start Rstudio application or type *R* in a terminal).
```R
install.packages(
  c('rzmq','repr','IRkernel','IRdisplay'),
  repos = c('http://irkernel.github.io/', getOption('repos'))
)
IRkernel::installspec()
```

* Installation: https://github.com/IRkernel/IRkernel
* Homepage: http://irkernel.github.io/

### Start IPython Notebook Server with R kernel

IPython vs Jupyter

```bash
ipython notebook
```

Choose *new* >> *R* from the dropdown list (top right corner)

## Activity 6: Setting up Git and GitHub

* Signup for an account in https://github.com/ (remember your github username and password)
* [Setup GitHub](https://help.github.com/articles/set-up-git/) (setup your local *git* installation for GitHub)
```bash
git config --global user.name "YourName"
git config --global user.email "YourGitHubEmailId"
```
* Learn the basics of git
  - [Set up Git in your machine](http://rogerdudler.github.io/git-guide/)
  - [Practise Git](https://try.github.io/levels/1/challenges/1)
* [GitHub Workflow using web-ui](https://help.github.com/articles/create-a-repo/)
* (Optional for Day 1) [Forking Repos](https://help.github.com/articles/fork-a-repo/) and [GitHub Fork Workflow](https://github.com/sevntu-checkstyle/sevntu.checkstyle/wiki/Development-workflow-with-Git:-Fork,-Branching,-Commits,-and-Pull-Request)
* (Optional for Day 1) [Set up SSH authentication](https://help.github.com/articles/generating-ssh-keys/)


## Activity 7: Literate Programming

### markdown basics

* [What is markdown?](https://help.github.com/articles/markdown-basics/)
* [GitHub Flavored Markdown](https://help.github.com/articles/github-flavored-markdown/) (used by IPython)

Here are some examples of markdown in action (edit this cell to see the raw text)
* Styling Text
    - *italicised*
    - **bold**
    - ***italicised and bold***
    - **styles _mixed_ together**
* Unordered lists
  - Nested Unordered Lists
  1. Ordered Lists
    1. Nested Lists
    2. More Nesting
      * More nested lists
* Writing Code Blocks
```R
x = 0
x = 2 + 2
hist(rnorm(1000))
```